<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/01_oneformer_evaluador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
"""
================================================================================
EVALUADOR ONEFORMER - NOTEBOOK 3
================================================================================
Sistema de evaluacion para modelos OneFormer con estructura estandarizada
para comparacion de modelos de segmentacion semantic/instance/panoptic.

Funcionalidades:
- Procesamiento incremental con checkpoint
- Gestion optima de memoria para Colab gratuito
- Guardado automatico despues de cada imagen
- Generacion de visualizaciones
- Mascaras organizadas por fotografia
- JSON con metricas de rendimiento
- Extraccion de scores de confianza por persona
- Deteccion multi-persona con instance segmentation
- Soporte para task_type configurable (semantic/instance/panoptic)

Entrada: Imagenes desde /TFM/0_Imagenes/
Salida: JSON, mascaras NPZ y visualizaciones en /TFM/2_Modelos/oneformer/

Autor: Jesus L.
Proyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion
Universidad: Universidad Oberta de Cataluna
Fecha: 2025
================================================================================
"""

'\n================================================================================\nEVALUADOR ONEFORMER - NOTEBOOK 3\n================================================================================\nSistema de evaluacion para modelos OneFormer con estructura estandarizada\npara comparacion de modelos de segmentacion semantic/instance/panoptic.\n\nFuncionalidades:\n- Procesamiento incremental con checkpoint\n- Gestion optima de memoria para Colab gratuito\n- Guardado automatico despues de cada imagen\n- Generacion de visualizaciones\n- Mascaras organizadas por fotografia\n- JSON con metricas de rendimiento\n- Extraccion de scores de confianza por persona\n- Deteccion multi-persona con instance segmentation\n- Soporte para task_type configurable (semantic/instance/panoptic)\n\nEntrada: Imagenes desde /TFM/0_Imagenes/\nSalida: JSON, mascaras NPZ y visualizaciones en /TFM/2_Modelos/oneformer/\n\nAutor: Jesus L.\nProyecto: TFM - Evaluacion Comparativa de Tecnicas de Segmentacion\nUniversi

In [3]:
import os
import gc
import time
import json
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, asdict, field

import numpy as np
import torch
import cv2
from PIL import Image
import matplotlib.pyplot as plt

from transformers import OneFormerProcessor, OneFormerForUniversalSegmentation

warnings.filterwarnings('ignore')

print("Librerias cargadas correctamente")


Librerias cargadas correctamente


In [4]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ModeloInfo:
    """Informacion de un modelo OneFormer"""
    nombre_hf: str
    nombre_corto: str
    tipo: str
    dataset: str
    arquitectura: str
    descripcion: str
    tareas_soportadas: List[str]

MODELOS_DISPONIBLES = {
    'coco_swin': ModeloInfo(
        nombre_hf="shi-labs/oneformer_coco_swin_large",
        nombre_corto="coco_swin",
        tipo='universal',
        dataset='COCO',
        arquitectura='Swin-Large',
        descripcion='OneFormer Universal - COCO dataset',
        tareas_soportadas=['semantic', 'instance', 'panoptic']
    ),
    'ade20k_swin': ModeloInfo(
        nombre_hf="shi-labs/oneformer_ade20k_swin_large",
        nombre_corto="ade20k_swin",
        tipo='universal',
        dataset='ADE20K',
        arquitectura='Swin-Large',
        descripcion='OneFormer Universal - ADE20K dataset',
        tareas_soportadas=['semantic', 'instance', 'panoptic']
    ),
    'ade20k_tiny': ModeloInfo(
        nombre_hf="shi-labs/oneformer_ade20k_swin_tiny",
        nombre_corto="ade20k_tiny",
        tipo='universal',
        dataset='ADE20K',
        arquitectura='Swin-Tiny',
        descripcion='OneFormer Universal - ADE20K Swin-Tiny',
        tareas_soportadas=['semantic', 'instance', 'panoptic']
    ),
    'coco_dinat': ModeloInfo(
        nombre_hf="shi-labs/oneformer_coco_dinat_large",
        nombre_corto="coco_dinat",
        tipo='universal',
        dataset='COCO',
        arquitectura='DiNAT-Large',
        descripcion='OneFormer Universal - COCO DiNAT',
        tareas_soportadas=['semantic', 'instance', 'panoptic']
    )
}

@dataclass
class ConfiguracionUmbrales:
    """Configuracion de umbrales de confianza"""
    nombre: str
    valores: List[float]
    descripcion: str
    task_type: str = "instance"

CONFIGURACIONES_UMBRALES = {
    'retrato_balanced': ConfiguracionUmbrales(
        nombre='retrato_balanced',
        valores=[0.4, 0.6, 0.75, 0.85],
        descripcion='Umbrales optimizados para fotografia de retrato',
        task_type='instance'
    ),
    'retrato_semantic': ConfiguracionUmbrales(
        nombre='retrato_semantic',
        valores=[0.4, 0.6, 0.75, 0.85],
        descripcion='Umbrales para segmentacion semantica',
        task_type='semantic'
    ),
    'retrato_panoptic': ConfiguracionUmbrales(
        nombre='retrato_panoptic',
        valores=[0.4, 0.6, 0.75, 0.85],
        descripcion='Umbrales para segmentacion panoptica',
        task_type='panoptic'
    )
}

@dataclass
class ConfiguracionEvaluacion:
    """Configuracion principal del sistema"""
    # Modelos a evaluar
    modelos_evaluar: List[str] = None

    # Configuraciones de umbrales a evaluar
    configs_umbrales: List[str] = None

    # Rutas
    ruta_base: Path = Path("/content/drive/MyDrive/TFM")
    ruta_imagenes: Path = None
    ruta_resultados: Path = None

    # Procesamiento
    max_dimension: int = 1920
    pausa_entre_imagenes: float = 2.0
    pausa_entre_modelos: float = 5.0

    # IDs de persona segun dataset
    person_id_coco: int = 0
    person_id_ade20k: int = 12

    # Threshold de confianza minimo
    min_confidence_score: float = 0.5

    def __post_init__(self):
        if self.ruta_imagenes is None:
            self.ruta_imagenes = self.ruta_base / "0_Imagenes"
        if self.ruta_resultados is None:
            self.ruta_resultados = self.ruta_base / "2_Modelos" / "oneformer"
        if self.modelos_evaluar is None:
            self.modelos_evaluar = list(MODELOS_DISPONIBLES.keys())
        if self.configs_umbrales is None:
            self.configs_umbrales = ['retrato_balanced']

In [5]:
# =============================================================================
# UTILIDADES
# =============================================================================

class Utilidades:
    """Funciones auxiliares"""

    @staticmethod
    def cargar_imagen(ruta: Path, max_dim: int = None) -> Tuple[Image.Image, Dict]:
        """Carga imagen con opcion de redimensionar"""
        img = Image.open(ruta).convert("RGB")
        dim_orig = img.size

        redim = False
        if max_dim and max(img.size) > max_dim:
            ratio = max_dim / max(img.size)
            new_size = tuple(int(d * ratio) for d in img.size)
            img = img.resize(new_size, Image.LANCZOS)
            redim = True

        info = {
            'original': {'width': dim_orig[0], 'height': dim_orig[1]},
            'procesada': {'width': img.size[0], 'height': img.size[1], 'redimensionada': redim}
        }
        return img, info

    @staticmethod
    def guardar_json(datos: Dict, ruta: Path):
        """Guarda JSON"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(ruta, 'w', encoding='utf-8') as f:
            json.dump(datos, f, indent=2, ensure_ascii=False)

    @staticmethod
    def guardar_mascara_npz(mascara: np.ndarray, ruta: Path):
        """Guarda mascara en NPZ comprimido"""
        ruta.parent.mkdir(parents=True, exist_ok=True)
        np.savez_compressed(ruta, mascara=mascara.astype(np.float32))

    @staticmethod
    def liberar_memoria():
        """Libera memoria GPU y RAM"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def obtener_memoria_gpu() -> Dict:
        """Info de memoria GPU"""
        if not torch.cuda.is_available():
            return {'disponible': False}
        return {
            'disponible': True,
            'nombre': torch.cuda.get_device_name(0),
            'asignada_mb': round(torch.cuda.memory_allocated(0) / 1024**2, 2),
            'reservada_mb': round(torch.cuda.memory_reserved(0) / 1024**2, 2)
        }


In [6]:
# =============================================================================
# GESTOR DE CHECKPOINT
# =============================================================================

class GestorCheckpoint:
    """Maneja checkpoint para retomar procesamiento"""

    def __init__(self, ruta: Path):
        self.ruta = ruta
        self.datos = self._cargar()

    def _cargar(self) -> Dict:
        if self.ruta.exists():
            with open(self.ruta, 'r') as f:
                return json.load(f)
        return {
            'completadas': [],
            'timestamp': datetime.now().isoformat()
        }

    def guardar(self):
        self.datos['timestamp'] = datetime.now().isoformat()
        self.ruta.parent.mkdir(parents=True, exist_ok=True)
        with open(self.ruta, 'w') as f:
            json.dump(self.datos, f, indent=2)

    def marcar_completada(self, nombre_imagen: str):
        if nombre_imagen not in self.datos['completadas']:
            self.datos['completadas'].append(nombre_imagen)
        self.guardar()

    def esta_completada(self, nombre_imagen: str) -> bool:
        return nombre_imagen in self.datos['completadas']


In [7]:
# =============================================================================
# VISUALIZADOR
# =============================================================================

class Visualizador:
    """Generacion de visualizaciones"""

    @staticmethod
    def crear_visualizacion(imagen: Image.Image, mascara: np.ndarray,
                          ruta_salida: Path, titulo: str = ""):
        """Crea visualizacion con 3 paneles"""
        # Crear directorio si no existe
        ruta_salida.parent.mkdir(parents=True, exist_ok=True)

        img_array = np.array(imagen)

        # Crear overlay
        overlay = img_array.copy()
        overlay[mascara > 0] = [0, 255, 0]
        result = cv2.addWeighted(img_array, 0.6, overlay, 0.4, 0)

        # Crear figura
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        axes[0].imshow(img_array)
        axes[0].set_title('Original')
        axes[0].axis('off')

        axes[1].imshow(mascara, cmap='gray')
        axes[1].set_title('Mascara')
        axes[1].axis('off')

        axes[2].imshow(result)
        axes[2].set_title('Overlay')
        axes[2].axis('off')

        if titulo:
            fig.suptitle(titulo, fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

In [8]:
# =============================================================================
# EVALUADOR ONEFORMER
# =============================================================================

class EvaluadorOneFormer:
    """Evaluador principal de OneFormer"""

    def __init__(self, config: ConfiguracionEvaluacion):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def get_person_id(self, dataset: str) -> int:
        """Retorna ID de persona segun dataset"""
        return self.config.person_id_coco if dataset == "COCO" else self.config.person_id_ade20k

    def procesar_imagen(self, imagen: Image.Image, procesador, modelo,
                       person_id: int, threshold: float, task_type: str) -> Tuple[np.ndarray, Dict]:
        """Procesa una imagen y retorna mascara + metadata"""

        # Preparar inputs
        task_inputs = procesador(
            imagen,
            task_inputs=[task_type],
            return_tensors="pt"
        )
        task_inputs = {k: v.to(self.device) for k, v in task_inputs.items()}

        # Inferencia
        with torch.no_grad():
            outputs = modelo(**task_inputs)

        # Post-procesamiento segun tipo de tarea
        if task_type == "semantic":
            predicted_map = procesador.post_process_semantic_segmentation(
                outputs,
                target_sizes=[imagen.size[::-1]]
            )[0]
            predicted_map = predicted_map.cpu().numpy()
            mascara_binaria = (predicted_map == person_id).astype(np.uint8)

            # Calcular estadisticas basicas
            num_pixeles_persona = np.sum(mascara_binaria)
            total_pixeles = mascara_binaria.size
            porcentaje_persona = (num_pixeles_persona / total_pixeles) * 100
            personas_detectadas = 1 if porcentaje_persona > (threshold * 0.5) else 0

            metadata = {
                'personas_detectadas': personas_detectadas,
                'pixeles_persona': int(num_pixeles_persona),
                'porcentaje_area': float(porcentaje_persona),
                'threshold_usado': threshold,
                'person_id_modelo': int(person_id),
                'task_type': task_type,
                'scores_promedio': None
            }

        elif task_type == "panoptic":
            result = procesador.post_process_panoptic_segmentation(
                outputs,
                target_sizes=[imagen.size[::-1]],
                threshold=threshold
            )[0]

            # Extraer segmentos de personas
            mascara_binaria = np.zeros(imagen.size[::-1], dtype=np.uint8)
            personas_detectadas = 0
            scores_personas = []
            pixeles_por_persona = []

            if len(result['segments_info']) > 0:
                segmentation = result['segmentation'].cpu().numpy()

                for segment_info in result['segments_info']:
                    if segment_info['label_id'] == person_id:
                        segment_id = segment_info['id']
                        score = segment_info.get('score', 1.0)

                        if score >= self.config.min_confidence_score:
                            mask_segment = (segmentation == segment_id).astype(np.uint8)
                            mascara_binaria = np.logical_or(mascara_binaria, mask_segment).astype(np.uint8)

                            personas_detectadas += 1
                            scores_personas.append(float(score))
                            pixeles_por_persona.append(int(np.sum(mask_segment)))

            num_pixeles_persona = np.sum(mascara_binaria)
            total_pixeles = mascara_binaria.size
            porcentaje_persona = (num_pixeles_persona / total_pixeles) * 100

            metadata = {
                'personas_detectadas': personas_detectadas,
                'pixeles_persona': int(num_pixeles_persona),
                'porcentaje_area': float(porcentaje_persona),
                'threshold_usado': threshold,
                'person_id_modelo': int(person_id),
                'task_type': task_type,
                'scores_personas': scores_personas,
                'scores_promedio': float(np.mean(scores_personas)) if scores_personas else None,
                'scores_min': float(np.min(scores_personas)) if scores_personas else None,
                'scores_max': float(np.max(scores_personas)) if scores_personas else None,
                'pixeles_por_persona': pixeles_por_persona,
                'min_confidence_usado': self.config.min_confidence_score
            }

        else:  # instance
            result = procesador.post_process_instance_segmentation(
                outputs,
                target_sizes=[imagen.size[::-1]],
                threshold=threshold
            )[0]

            # Extraer segmentos de personas
            mascara_binaria = np.zeros(imagen.size[::-1], dtype=np.uint8)
            personas_detectadas = 0
            scores_personas = []
            pixeles_por_persona = []

            if len(result['segments_info']) > 0:
                segmentation = result['segmentation'].cpu().numpy()

                for segment_info in result['segments_info']:
                    if segment_info['label_id'] == person_id:
                        segment_id = segment_info['id']
                        score = segment_info.get('score', 1.0)

                        if score >= self.config.min_confidence_score:
                            mask_segment = (segmentation == segment_id).astype(np.uint8)
                            mascara_binaria = np.logical_or(mascara_binaria, mask_segment).astype(np.uint8)

                            personas_detectadas += 1
                            scores_personas.append(float(score))
                            pixeles_por_persona.append(int(np.sum(mask_segment)))

            num_pixeles_persona = np.sum(mascara_binaria)
            total_pixeles = mascara_binaria.size
            porcentaje_persona = (num_pixeles_persona / total_pixeles) * 100

            metadata = {
                'personas_detectadas': personas_detectadas,
                'pixeles_persona': int(num_pixeles_persona),
                'porcentaje_area': float(porcentaje_persona),
                'threshold_usado': threshold,
                'person_id_modelo': int(person_id),
                'task_type': task_type,
                'scores_personas': scores_personas,
                'scores_promedio': float(np.mean(scores_personas)) if scores_personas else None,
                'scores_min': float(np.min(scores_personas)) if scores_personas else None,
                'scores_max': float(np.max(scores_personas)) if scores_personas else None,
                'pixeles_por_persona': pixeles_por_persona,
                'min_confidence_usado': self.config.min_confidence_score
            }

        return mascara_binaria, metadata

    def evaluar_modelo_con_config_umbrales(self, modelo_key: str,
                                          config_umbral: str,
                                          imagenes: List[Path]):
        """Evalua un modelo con una configuracion de umbrales"""

        modelo_info = MODELOS_DISPONIBLES[modelo_key]
        umbrales = CONFIGURACIONES_UMBRALES[config_umbral]
        task_type = umbrales.task_type

        print(f"\n{'='*80}")
        print(f"Modelo: {modelo_info.nombre_corto} ({modelo_info.dataset})")
        print(f"Config umbrales: {config_umbral} - {umbrales.valores}")
        print(f"Task type: {task_type}")
        print(f"{'='*80}")

        # Cargar modelo
        print(f"\nCargando modelo {modelo_info.nombre_hf}...")
        tiempo_inicio_carga = time.time()

        procesador = OneFormerProcessor.from_pretrained(modelo_info.nombre_hf)
        modelo = OneFormerForUniversalSegmentation.from_pretrained(
            modelo_info.nombre_hf
        ).to(self.device)
        modelo.eval()

        print(f"Modelo cargado en {time.time() - tiempo_inicio_carga:.1f}s")
        print(f"Dispositivo: {self.device}")

        memoria_info = Utilidades.obtener_memoria_gpu()
        if memoria_info['disponible']:
            print(f"GPU: {memoria_info['nombre']}")
            print(f"Memoria asignada: {memoria_info['asignada_mb']} MB")

        # Obtener ID de persona
        person_id = self.get_person_id(modelo_info.dataset)

        # Procesar cada threshold
        for umbral in umbrales.valores:
            config_id = f"{modelo_info.nombre_corto}_{task_type}_t{umbral:.2f}"
            dir_config = self.config.ruta_resultados / config_id

            dir_mascaras = dir_config / "mascaras"
            dir_json = dir_config / "json"
            dir_viz = dir_config / "visualizaciones"
            checkpoint_path = dir_config / "checkpoint.json"

            checkpoint = GestorCheckpoint(checkpoint_path)

            print(f"\n--- Threshold: {umbral} ---")

            imgs_procesar = [img for img in imagenes
                           if not checkpoint.esta_completada(img.name)]

            if not imgs_procesar:
                print(f"Todas las imagenes ya procesadas con threshold {umbral}")
                continue

            print(f"Imagenes a procesar: {len(imgs_procesar)}/{len(imagenes)}")

            tiempo_inicio = time.time()

            for i, img_path in enumerate(imgs_procesar, 1):
                try:
                    print(f"\n[{i}/{len(imgs_procesar)}] {img_path.name}")

                    # Cargar y procesar imagen
                    imagen, info_dim = Utilidades.cargar_imagen(
                        img_path,
                        self.config.max_dimension
                    )

                    if info_dim['procesada']['redimensionada']:
                        print(f"  Redimensionada a {imagen.size}")

                    # Inferencia
                    mascara, metadata = self.procesar_imagen(
                        imagen, procesador, modelo, person_id, umbral, task_type
                    )

                    # Re-escalar mascara si necesario
                    if info_dim['procesada']['redimensionada']:
                        mascara = cv2.resize(
                            mascara,
                            (info_dim['original']['width'], info_dim['original']['height']),
                            interpolation=cv2.INTER_NEAREST
                        )
                        # Recargar imagen original para visualizacion
                        imagen_orig = Image.open(img_path).convert("RGB")
                    else:
                        imagen_orig = imagen

                    # Guardar resultados
                    nombre_base = img_path.stem

                    # Mascara NPZ
                    ruta_mascara = dir_mascaras / f"{nombre_base}.npz"
                    Utilidades.guardar_mascara_npz(mascara, ruta_mascara)

                    # Metadata JSON
                    metadata_completa = {
                        'nombre_imagen': img_path.name,
                        'modelo': modelo_info.nombre_corto,
                        'dataset': modelo_info.dataset,
                        'arquitectura': modelo_info.arquitectura,
                        'threshold': umbral,
                        'dimensiones': info_dim,
                        'resultados': metadata,
                        'timestamp': datetime.now().isoformat()
                    }

                    ruta_json = dir_json / f"{nombre_base}.json"
                    Utilidades.guardar_json(metadata_completa, ruta_json)

                    # Visualizacion
                    ruta_viz = dir_viz / f"{nombre_base}.png"
                    titulo = f"{modelo_info.dataset} | {modelo_info.arquitectura} | {task_type} | t={umbral}"
                    Visualizador.crear_visualizacion(
                        imagen_orig, mascara, ruta_viz, titulo
                    )

                    # Marcar completada
                    checkpoint.marcar_completada(img_path.name)

                    print(f"  Completada - {metadata['personas_detectadas']} personas")

                except Exception as e:
                    print(f"  ERROR: {str(e)}")
                    import traceback
                    traceback.print_exc()

                finally:
                    Utilidades.liberar_memoria()
                    if i < len(imgs_procesar):
                        time.sleep(self.config.pausa_entre_imagenes)

            tiempo_total = time.time() - tiempo_inicio
            print(f"\nThreshold {umbral} completado en {tiempo_total:.1f}s")

        # Limpiar modelo
        del procesador
        del modelo
        Utilidades.liberar_memoria()

        print(f"\nModelo {modelo_info.nombre_corto} completado")

    def ejecutar(self):
        """Ejecuta evaluacion completa"""

        print(f"\n{'='*80}")
        print("EVALUADOR ONEFORMER - NOTEBOOK 3")
        print(f"{'='*80}")
        print(f"Modelos: {len(self.config.modelos_evaluar)}")
        print(f"Configuraciones de umbrales: {len(self.config.configs_umbrales)}")
        for config_name in self.config.configs_umbrales:
            umbrales = CONFIGURACIONES_UMBRALES[config_name]
            print(f"  - {config_name}: {umbrales.valores}")
        print(f"Imagenes: {self.config.ruta_imagenes}")

        # Cargar imagenes
        imagenes = sorted(self.config.ruta_imagenes.glob("*.jpg"))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.JPG")))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.png")))
        imagenes.extend(sorted(self.config.ruta_imagenes.glob("*.PNG")))
        imagenes = sorted(set(imagenes))

        print(f"Total imagenes encontradas: {len(imagenes)}")

        if not imagenes:
            print("ERROR: No se encontraron imagenes")
            return

        # Evaluar cada combinacion
        total_combinaciones = len(self.config.modelos_evaluar) * len(self.config.configs_umbrales)
        combinacion_actual = 0

        for modelo_key in self.config.modelos_evaluar:
            for config_umbral in self.config.configs_umbrales:
                combinacion_actual += 1

                print(f"\n{'#'*80}")
                print(f"COMBINACION {combinacion_actual}/{total_combinaciones}")
                print(f"{'#'*80}")

                self.evaluar_modelo_con_config_umbrales(modelo_key, config_umbral, imagenes)

                if combinacion_actual < total_combinaciones:
                    print(f"\nPausa entre configuraciones: {self.config.pausa_entre_modelos}s")
                    time.sleep(self.config.pausa_entre_modelos)

        print(f"\n{'='*80}")
        print("EVALUACION COMPLETADA")
        print(f"{'='*80}")
        print(f"Resultados en: {self.config.ruta_resultados}")

In [9]:
# =============================================================================
# EJECUCION
# =============================================================================

def main():
    """Funcion principal"""

    # Montar Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
    except Exception as e:
        print(f"No se pudo montar Drive: {e}")
        print("Continuando sin Drive (probablemente no estas en Colab)")

    # Configurar
    config = ConfiguracionEvaluacion(
        # Seleccionar modelos
        modelos_evaluar=None,  # None = todos

        # Seleccionar configuraciones de umbrales
        # Cada configuracion define su propio task_type
        configs_umbrales=[
            'retrato_balanced',   # instance
            # 'retrato_semantic',   # semantic
            # 'retrato_panoptic'    # panoptic
        ],

        # Opciones
        max_dimension=1920,
        pausa_entre_imagenes=2.0,
        pausa_entre_modelos=5.0,
        min_confidence_score=0.5
    )

    # Ejecutar
    evaluador = EvaluadorOneFormer(config)
    evaluador.ejecutar()

In [ ]:
if __name__ == "__main__":
    main()

Mounted at /content/drive

EVALUADOR ONEFORMER - NOTEBOOK 3
Modelos: 4
Configuraciones de umbrales: 1
  - retrato_balanced: [0.4, 0.6, 0.75, 0.85]
Imagenes: /content/drive/MyDrive/TFM/0_Imagenes
Total imagenes encontradas: 20

################################################################################
COMBINACION 1/4
################################################################################

Modelo: coco_swin (COCO)
Config umbrales: retrato_balanced - [0.4, 0.6, 0.75, 0.85]
Task type: instance

Cargando modelo shi-labs/oneformer_coco_swin_large...


preprocessor_config.json: 0.00B [00:00, ?B/s]

coco_panoptic.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/880M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/879M [00:00<?, ?B/s]

Modelo cargado en 84.3s
Dispositivo: cuda
GPU: Tesla T4
Memoria asignada: 906.23 MB

--- Threshold: 0.4 ---
Imagenes a procesar: 20/20

[1/20] _DSC0023.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[2/20] _DSC0036.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[3/20] _DSC0071.jpg
  Redimensionada a (1279, 1919)
  Completada - 1 personas

[4/20] _DSC0084.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[5/20] _DSC0119.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[6/20] _DSC0139.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[7/20] _DSC0143.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[8/20] _DSC0147.jpg
  Redimensionada a (1280, 1919)
  Completada - 3 personas

[9/20] _DSC0281.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[10/20] _DSC0283.jpg
  Redimensionada a (1280, 1920)
  Completada - 1 personas

[11/20] _DSC0411.jpg
  Redimensionada a (1280, 1920)
  Completada - 3 per

KeyboardInterrupt: 